In [1]:
from pyspark import SparkContext, SparkConf, SparkFiles
import string

In [2]:
def mapper1(line):
    punc = string.punctuation    
    tmp, i, j, value = '', '', '', ''
    flag = -1
    
    for word in line:
        if word.isdigit():
            tmp += word
        elif word in punc:
            if flag == 0:
                i = tmp
            elif flag == 1:
                j = tmp
            flag +=1
            tmp = ''
            
    value = tmp

    if line[0] == 'M':
        return j, (line[0], i, value)
    else:
        return i, (line[0], j, value)
    
    
def mapper2(line):
    i = int(line[1][0][1])
    j = int(line[1][1][1])
    mul = int(line[1][0][2]) * int(line[1][1][2])
    return (i, j), mul

def reducer(a, b):
    return a + b

In [3]:
def main():
    path = "500input.txt"
    file = sc.textFile(path)
    M = file.filter(lambda x: 'M' in x)
    N = file.filter(lambda x: 'N' in x)

    M1 = M.map(mapper1)
    N1 = N.map(mapper1)
    M1 = M1.join(N1)


    Cal = M1.map(mapper2)
    Ans = Cal.reduceByKey(reducer).sortByKey().collect()
    #for a in Ans:
        #print(a)

    outfile = open('Outputfile.txt','w')
    for a in Ans:
        s = str(a[0][0]) + ',' + str(a[0][1]) + ',' + str(a[1]) + '\n'
        outfile.write(s)

if __name__ == '__main__':
    main()

In [4]:
sc.stop()

## MDA_HW1 Matrix Multiplication

在這次的作業中，我們要對500X500的大矩陣使用MapReduce的技巧來做矩陣乘法。

對於乘法的結果矩陣 R，每一項的算式如下，其中i的大小是M矩陣的row大小，j則是N矩陣的column大小。

$r_{ij} = \sum m_{ik}n_{kj}$

讀進檔案時，先用RDD的運算—filter將M、N兩個矩陣分開，方便之後的計算。

我總共使用了兩個mapper function和一個reducer funcition，其中mapper1會分別對兩個矩陣的資料做，做完之後就會將兩筆資料join在一起，繼續往下做。

    
* <font color = blue size=4> mapper1 </font>
 
    在mapper1中，首先先將從txt檔讀出來的資料整理成我們所希望的樣子：
    $(key, value) = (k, (M, i, m_{ik}))$  或  $((k, (N, j, m_{kj}))$
    
    因為不能用list還有和list相關的運算，所以很土法煉鋼的把讀進來的string分成數字。
    
    分數字的方法是，在逗號之前的數字（char）都會是同一個數字（int），比如說12在string中會分別被讀成1和2兩個char，但我們要的是12。因此用$isdigit$判斷讀到的字是不是數字，若是的話就和之前讀到的串在一起，直到遇到逗號分隔，把這個串好的字保留下來。用flag判斷現在取出的數字是row/column index或是value。
    
    分別獲得矩陣名稱、index和該位置的值後，就能將這些值以我們要的key-value方式回傳做map。
    
    
* <font color = blue size=4> mapper2 </font>

    在將兩筆資料join $（ M.join(N) )$ 起來後，key值相同的項就會被放到一起，資料的型態會變成像這樣：
    $(key, (value, value) )$，且對於同樣的key值會有$i * j$ 項（行）。
    
    其中每個value都是一個tuple：
    $(k, (M, i, m_{ik}))$ 或 $(k, (N, j, n_{kj}))$
    這樣對於每一項（行），我們就可以取出兩個矩陣中的兩個值做相乘 $m_{ik}n_{kj}$。
    最後把$(i, j)$當作key，相乘的結果 $m_{ik}n_{kj}$ 當作value組成新的pair（為了最後的排序，三個數字都轉成int型別，最後要寫入檔案時再改回），做第二次的map。
          
          
* <font color = blue size=4> reducer </font>  

    經過mapper2之後資料的型態為 $ ((i, j), value) $，這時只要使用$reduceByKey$，並將reducer function定義為相加，將相同key的value加起來，就能獲得$r_{ij}$了！
    

    
做完reduce後，順序可能不會像原本一樣由小到大排得好好的，所以先用$sortByKey()$對資料先做排序，最後用$collect()$把RDD轉換成list，再將裡面的數字轉換成字串，寫到txt檔中。
    
    
#### Reference:
* https://www.tutorialspoint.com/pyspark/pyspark_rdd.htm
* https://www.learncodewithmike.com/2019/12/python-lambda-functions.html
* https://codertw.com/%E7%A8%8B%E5%BC%8F%E8%AA%9E%E8%A8%80/711947/
